# Explore the dataset

In [ ]:
%matplotlib inline
import os
import matplotlib
import matplotlib.pyplot as plt
from interface import dataframe
import argparse
import glob
import shutil

These should always stay the same.

In [ ]:
%cd '/ocean/projects/hum180001p/shared/src/shuffler'
rootdir='/ocean/projects/hum180001p/shared/data/'
campaign_dir='/ocean/projects/hum180001p/shared/databases/'

File paths for latest versions of databases.

In [ ]:
campaign3_db    = campaign_dir + 'campaign3/campaign3-6Kx4K.v6.db'
campaign4_db    = campaign_dir + 'campaign4/campaign4-6Kx4K.v10.db'
campaign5_db    = campaign_dir + 'campaign5/campaign5-1800x1200.v4.db'
campaign6_db    = campaign_dir + 'campaign6/campaign6-6Kx4K.v7.db'
campaign3to5_db = campaign_dir + 'campaign5/campaign3to5-6Kx4K.v6.db'
campaign3to6_db = campaign_dir + 'campaign6/campaign3to6-6Kx4K.v7.db'
campaign3to6_cropped_db = campaign_dir + 'campaign6/crops/campaign3to6-6Kx4K.v7-croppedStamps.db'

# Check that the files exist.
assert os.path.exists(campaign3_db), campaign3_db
assert os.path.exists(campaign4_db), campaign4_db
assert os.path.exists(campaign5_db), campaign5_db
assert os.path.exists(campaign6_db), campaign6_db
assert os.path.exists(campaign3to5_db), campaign3to5_db
assert os.path.exists(campaign3to6_db), campaign3to6_db
assert os.path.exists(campaign3to6_cropped_db), campaign3to6_cropped_db

### Explore cropped stamps 

Load the database.

In [ ]:
df = dataframe.Dataframe(in_db_path=campaign3to6_cropped_db,  rootdir=rootdir)

Display random images.

In [ ]:
df.displayImagesPlt(limit=8, with_objects=True, with_imagefile=True, ncols=4, shuffle=True)
plt.rcParams["figure.figsize"] = [20,40]

### Explore campaigns 3-6

Load the databases.

In [ ]:
# campaign6 alone.
df6    = dataframe.Dataframe(in_db_path=campaign6_db, rootdir=rootdir)

# campaigns 3 to 6.
df3to6 = dataframe.Dataframe(in_db_path=campaign3to6_db, rootdir=rootdir)

How many instances and distinct names are in **campaign 6** alone. These are new stamps in this campaign.

In [ ]:
df6.cursor.execute("SELECT COUNT(1) FROM objects WHERE name NOT LIKE '%page%'")
print('stamp instances in campaign 6: ', df6.cursor.fetchone()[0])

df6.cursor.execute("SELECT COUNT(DISTINCT(name)) FROM objects WHERE name NOT LIKE '%page%'")
print('stamp names in campaign 6:     ', df6.cursor.fetchone()[0])

How many instances and distinct names are in all **campaigns 3 to 6**.

In [ ]:
df3to6.cursor.execute("SELECT COUNT(1) FROM objects WHERE name NOT LIKE '%page%'")
print('stamp instances in campaigns 3-6: ', df3to6.cursor.fetchone()[0])

df3to6.cursor.execute("SELECT COUNT(DISTINCT(name)) FROM objects WHERE name NOT LIKE '%page%'")
print('stamp names in campaigns 3-6:     ', df3to6.cursor.fetchone()[0])

Display a random image.

In [ ]:
plt.rcParams["figure.figsize"] = [20,20]
df3to6.displayImagesPlt(limit=1, with_objects=True, with_imagefile=True, ncols=1, shuffle=True)

Plot the histogram of all classes for the paper, split across campaigns.

In [ ]:
matplotlib.rc('legend', fontsize=30, handlelength=2)
matplotlib.rc('ytick', labelsize=30)
from tools import PlotObjectNameHistograms

In [ ]:
args = argparse.Namespace(rootdir=rootdir, 
                          db_paths=[campaign3_db, campaign4_db, campaign5_db, campaign6_db],
                          out_plot_path=campaign_dir+'campaign6/visualization/campaign3to6.count.v7.png',
                          where_objects='name NOT LIKE "page%" AND name != "??"',
                          fig_width=50, 
                          fig_height=7, 
                          no_xticks=True, 
                          legend_entries=["cycle 1", "cycle 2", "cycle 3", "cycle 4"],
                          show=True,
                          ylog=True,
                          at_least=2,
                          fontsize=30)
PlotObjectNameHistograms.plot_object_name_histograms(args)

In [ ]:
args = argparse.Namespace(rootdir=rootdir, 
                          db_paths=[campaign3_db, campaign4_db, campaign5_db, campaign6_db],
                          out_plot_path=campaign_dir+'campaign6/visualization/campaign3to6.count.v7.ticks.png',
                          where_objects='name NOT LIKE "page%" AND name != "??"',
                          fig_width=13, 
                          fig_height=7, 
                          no_xticks=False, 
                          legend_entries=["cycle 1", "cycle 2", "cycle 3", "cycle 4"],
                          show=True,
                          ylog=False,
                          at_least=20,
                          fontsize=10)
PlotObjectNameHistograms.plot_object_name_histograms(args)

How many subfolders (e.g. `"01 (1427)"`) have been sampled from.

In [ ]:
# Count the number of subfolders in campaign6.
df6.cursor.execute("SELECT COUNT(DISTINCT(SUBSTR(imagefile,0,28))) FROM images")
print ('Number of subfolders in campaign 6 alone:  ', df6.cursor.fetchone()[0])

# Count the number of subfolders in campaign3to6.
df3to6.cursor.execute("SELECT COUNT(DISTINCT(SUBSTR(imagefile,0,28))) FROM images")
print ('Number of subfolders in campaigns 3 to 6:  ', df3to6.cursor.fetchone()[0])

# Count the total number of subfolders.
total = len(glob.glob(rootdir+"original_dataset/*"))
print ('Total number of subfolders:                ', total)

How many stamps in this campaign
1. from new classes 
2. from classes that were not in training data for classification before.

In [ ]:
df3to6.cursor.execute('SELECT COUNT(1) FROM objects WHERE name NOT LIKE "page%%" AND name != "??"')
num_all = df3to6.cursor.fetchone()[0]
df3to6.cursor.execute('ATTACH "%s" AS ref;' % campaign3to5_db)
df3to6.cursor.execute('SELECT COUNT(1) FROM objects WHERE name NOT LIKE "page%%" AND name != "??" AND name IN (SELECT DISTINCT(name) FROM ref.objects);')
num_from_known_classes = df3to6.cursor.fetchone()[0]
df3to6.cursor.execute('SELECT COUNT(1) FROM objects WHERE name NOT LIKE "page%%" AND name != "??" AND name IN (SELECT DISTINCT(name) FROM ref.objects GROUP BY name HAVING COUNT(1) >= 5);')
num_from_classes_with_over_5 = df3to6.cursor.fetchone()[0]
df3to6.cursor.execute('DETACH DATABASE ref;')
print ('Stamps from new classes in the latest campaign:        %.1f%%' % (100 - num_from_known_classes / num_all * 100))
print ('Stamps from classes that had at most 4 stamps before:  %.1f%%' % (100 - num_from_classes_with_over_5 / num_all * 100))

Distribution of the most common stamps by decade.

In [ ]:
plt.rcParams["figure.figsize"] = [5,5]

shutil.copyfile(campaign3to6_db, campaign_dir + 'temp.db')

with open(campaign_dir + 'info_years.txt') as f:
  lines = f.readlines()

df = dataframe.Dataframe(campaign_dir + 'temp.db')
for line in lines:
    folder, _, year1, year2 = line.strip().split('\t')
    year = (int(year1) + int(year2)) / 2
    decade = int(year / 10) * 10
    df.cursor.execute('SELECT objectid FROM objects WHERE imagefile LIKE "%%/%s%%"' % folder)
    objectids = df.cursor.fetchall()
    #print (folder, decade, len(objectids))
    for objectid, in objectids:
      df.cursor.execute('INSERT INTO properties(objectid,key,value) VALUES (?,\'decade\',?)', (objectid, decade))

df.sql(sql="DELETE FROM objects WHERE name IN (SELECT DISTINCT(name) FROM objects GROUP BY name HAVING COUNT(1) < 70)")
df.sql(sql="DELETE FROM objects WHERE name LIKE '%page%' OR name LIKE '%??%'")
df.plotHistogram(
    sql_stacked="SELECT value,name FROM properties JOIN objects ON properties.objectid = objects.objectid WHERE key='decade'",
    xlabel="decade",
    colormap="tab20",
    out_path=campaign_dir + "campaign6/visualization/campaign3to6-6Kx4K.v4.decade.png")
df.close()

os.remove(campaign_dir + 'temp.db')